In [1]:
from arcgis.gis import GIS
from azure.storage.queue import QueueService
import json

import sys
sys.path.append('../src')
sys.path.append('../')

from reach_tools import ReachFeatureLayer
import config

In [2]:
queue_conn_str = 'DefaultEndpointsProtocol=https;AccountName=whitewaterstor;AccountKey=KGSpleClj9AXohX0/uuYCe94cmwExJqfO0nUML7Ijb2eaWqcX1vRfws0+JiJQK+IpSkslWiznznXxoSrhM+yyg==;EndpointSuffix=core.windows.net'
update_queue_name = 'reach-update-queue'

In [3]:
%%time
# create the connection to the GIS to get access
gis = GIS(username=config.arcgis_username, password=config.arcgis_password)

# create a layer instance to query and get all existing features
lyr_centroid = ReachFeatureLayer(config.url_reach_centroid, gis)

# creat a connection to the Azure Queue
queue = QueueService(connection_string=queue_conn_str)

# query the feature service to get all existing features - notably the reach ids
reach_id_fs = lyr_centroid.query(out_fields='reach_id', return_geometry=False)

# iterate the features and load up the queue with each respective feature
for feature in reach_id_fs.features:
    
    # convert the feature json to a string
    feature_str = json.dumps(feature.as_dict)
    
    # push the feature to the queue
    queue.put_message(update_queue_name, feature_str)

CPU times: user 22 s, sys: 1.73 s, total: 23.7 s
Wall time: 1min 53s
